In [ ]:
from src.backtesting_program import BacktestingPorgram
from src.checker import Checker
import pandas as pd
from config import settings

In [ ]:
data = pd.read_csv("data.csv")


In [3]:
settings.LOGS_FILE_PATH = "output/logs.csv"
settings.EXECUTED_TRADES_FILE_PATH = "output/executed_trades.json"

checker = Checker(
  active_checkers={
    "when_trade_is_opened" : {
      "global": [
        "end_time_checker"
      ],
      "to_close_sell": [
        "updating_sell_stop_loss_checker",
        "to_close_sell_trade_1_checker",
        "to_close_sell_trade_2_checker",
        "to_close_sell_trade_3_checker"
      ],
      "to_close_buy": [
        "updating_buy_stop_loss_checker",
        "to_close_buy_trade_1_checker",
        "to_close_buy_trade_2_checker",
        "to_close_buy_trade_3_checker"
      ]
    },
    "when_trade_is_not_opened" : {
      "global": [
      "start_time_checker",
      "no_more_trades_time_checker",
      ],
      # if checker returns "SKIP", program skips all "to_open_sell" checkers and goes to "to_open_buy" checkers
      "to_open_sell": [
        "if_sell_was_executed_once_this_day",
        "to_sell_order_1_checker",
      ],
      "to_open_buy": [
      "if_buy_was_executed_once_this_day",
      "to_buy_order_1_checker",
      ]
    }
  },
  data=data,
  params={"start_time": "06:00", "end_time": "23:00", "no_more_trades_time": "21:30"},
)

program = BacktestingPorgram(
  historical_data=data,
  checker=checker,
)

program.start()

In [4]:
program.summary

,Opened Trades,Win Ratio,Average R:R,P/L,Consecutive Losses,Losses Quantity,BE Quantity,Win Quantity
0,14,50.0%,1:4.17,1.15%,4,7,0,7


In [8]:
# example of accessing class vars
program.executed_trades[2].logs

,GmtTime,Log
0,2023-06-02 20:00:00+00:00,Trade BUY 1% was opened on the price 1948.28.\...
1,2023-06-02 20:15:00+00:00,Trade BUY 1% was closed on the price 1948.36.\...
